In [ ]:
#THIS notebook generates results with my mutation attribution script
#It then uses these results to annotate hypermutated cases (features some complexities like defining MSI cases and assigning MSI muts)

In [196]:
import pandas as pd
import os
import sys

sys.path.append('/Users/friedman/Desktop/mnt/juno/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils
import numpy as np
import attribute_mutations_to_signatures
import clonality_analysis_util

pathPrefix = '/Users/friedman/Desktop/mnt'
from collections import Counter

In [2]:
allImpactMutsMaf = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/data_mutations_extended_annotated_sigContext_nov19_2019.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,17,18,40,41,43,44,71,72,108,124,125,128,129,130,131,132,133,134,135,137,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
#allHypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(hypermutantIdDir= pathPrefix + '/juno/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')
#hypermutantMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(allHypermutantIds)]
signatures = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/impactSignatureCalls_Nov20_2019.tsv')
hypermutantOnlySignatures = signatures[signatures['Tumor_Sample_Barcode'].isin(allHypermutantIds)]

In [228]:
hypermutantMaf = allImpactMutsMaf[allImpactMutsMaf['Tumor_Sample_Barcode'].isin(allHypermutantIds)]

In [35]:
hypermutantOnlySignatures.to_csv(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/hypermutants_only_signatures_nov19.txt', index=False, sep='\t')

In [36]:
mutAttributionDf = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/sigAttribution_hypermutators.tsv')

In [230]:
reload(attribute_mutations_to_signatures)
hypermutantMaf = attribute_mutations_to_signatures.attribute_mutations_to_signatures(hypermutantMaf, mutAttributionDf)


attributing mutations


In [ ]:
#Throw out msi indeterminate?-->Currently indeterminate are marked as msi as well
msiCases = get_gene_and_cohort_list_utils.get_msi_cases(msiInfoFilePath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mutations_TMB_and_MSI_stats.txt', msiScoreThresh=3)
hypermutantMaf['hypermutationInduced'] = hypermutantMaf.apply(lambda row:
        'hyperInduced' if row['Variant_Type'] in set(['INS', 'DEL']) and row['Tumor_Sample_Barcode'] in msiCases
                                                              
                    #TODO there is an error that is messing up POLE attribution
        else 'hyperInduced' if row['Variant_Type'] == 'SNP' and len(str(row['mutAttribution'])) > 3 and 'Signature.1' not in str(row['mutAttribution']).split('|')
        #else 'hyperInduced' if row['Variant_Type'] == 'SNP' and len(str(row['mutAttribution'])) > 3
        
                                                              
        else 'unclear' if row['Variant_Type'] == 'SNP' and row['mutAttribution'] != None and 'Signature.1' in str(row['mutAttribution']) and str(row['mutAttribution']) != 'Signature.1'
        else 'notHyperAttributable' ,axis=1)


In [ ]:
hypermutantMaf['geneAndCause'] = hypermutantMaf.apply(lambda row: str(row['Hugo_Symbol']) + '_' + row['hypermutationInduced'], axis=1)


In [234]:
hypermutantMaf.to_csv(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/mafWithMutationAttribution.tsv', index=False, sep='\t')

In [248]:
#
##
#####
############
#################
############
#######
####
#
#CLONALITY and attribution
#clonalityAnnotatedMaf = pd.read_table(pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/filteredMaf_Nov19_2019_hypermutantOnly_withCNCFAnnotation.maf')
mutAttributionAnnotatedMaf = hypermutantMaf

In [249]:
clonalityAnnotatedMaf['varId'] = clonalityAnnotatedMaf.apply(lambda row: str(row['Chromosome']) + '_' 
+ str(row['Start_Position']) + '_' + str(row['Reference_Allele']) + '->' + str(row['Tumor_Seq_Allele2']), axis=1)
mutAttributionAnnotatedMaf['varId'] = mutAttributionAnnotatedMaf.apply(lambda row: str(row['Chromosome']) + '_' 
+ str(row['Start_Position']) + '_' + str(row['Reference_Allele']) + '->' + str(row['Tumor_Seq_Allele2']), axis=1)
#clonalityAnnotatedMaf['ccf_Mcopies_lower']

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [250]:
ccfDict = dict(zip(clonalityAnnotatedMaf['varId'], clonalityAnnotatedMaf['ccf_Mcopies']))
mutAttributionAnnotatedMaf['ccf'] = mutAttributionAnnotatedMaf['varId'].apply(lambda x: ccfDict[x] if x in ccfDict else None)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [261]:
cancerTypeDict = analysis_utils.get_cancer_type_information(cancerTypeDfPath = pathPrefix + '/juno/work/taylorlab/friedman/myAdjustedDataFiles/cancerTypeInfo_asOfNov192019.txt', mode='pid')
mutAttributionAnnotatedMaf['pid'] = mutAttributionAnnotatedMaf['Tumor_Sample_Barcode'].apply(lambda x: x[:9])
mutAttributionAnnotatedMaf['cancerType'] = mutAttributionAnnotatedMaf['pid'].apply(lambda x: cancerTypeDict[x] if x in cancerTypeDict else None)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [287]:
#np.nanmean()
"""print np.nanmean(mutAttributionAnnotatedMaf[mutAttributionAnnotatedMaf['hypermutationInduced'] == 'hyperInduced']['ccf'])
print np.nanmean(mutAttributionAnnotatedMaf[mutAttributionAnnotatedMaf['hypermutationInduced'] == 'unclear']['ccf'])
print np.nanmean(mutAttributionAnnotatedMaf[mutAttributionAnnotatedMaf['hypermutationInduced'] == 'notHyperAttributable']['ccf'])

print 'oncogenic'
print np.nanmean(mutAttributionAnnotatedMaf[(mutAttributionAnnotatedMaf['oncogenic'].notnull()) & 
    (mutAttributionAnnotatedMaf['hypermutationInduced'] == 'hyperInduced')]['ccf'])
print np.nanmean(mutAttributionAnnotatedMaf[(mutAttributionAnnotatedMaf['oncogenic'].notnull()) & 
    (mutAttributionAnnotatedMaf['hypermutationInduced'] == 'unclear')]['ccf'])
print np.nanmean(mutAttributionAnnotatedMaf[(mutAttributionAnnotatedMaf['oncogenic'].notnull()) & 
    (mutAttributionAnnotatedMaf['hypermutationInduced'] == 'notHyperAttributable')]['ccf'])
    """
######################
###############
#########

for i in mutAttributionAnnotatedMaf[(mutAttributionAnnotatedMaf['oncogenic'].notnull()) & 
    (mutAttributionAnnotatedMaf['Hugo_Symbol'] == 'PTEN') &
    (mutAttributionAnnotatedMaf['HGVSp_Short'] == 'p.R130Q') &                                 
    (mutAttributionAnnotatedMaf['cancerType'] == 'Endometrial Cancer')]['Tumor_Sample_Barcode']:
    print i

#Counter(mutAttributionAnnotatedMaf[mutAttributionAnnotatedMaf['Hugo_Symbol'] == 'PTEN']['HGVSp_Short']).most_common(10)

#np.nanmean(mutAttributionAnnotatedMaf[(mutAttributionAnnotatedMaf['oncogenic'].notnull()) & 
#    (mutAttributionAnnotatedMaf['hypermutationInduced'] == 'notHyperAttributable') &
#    (mutAttributionAnnotatedMaf['cancerType'] == 'Glioma')]['ccf'])

P-0011569-T01-IM5
P-0011570-T01-IM5
P-0012113-T01-IM5
P-0015626-T01-IM6
P-0016099-T01-IM6
P-0016825-T01-IM6
P-0019649-T01-IM6
P-0019658-T01-IM6
P-0020754-T01-IM6
P-0021283-T01-IM6
P-0021856-T01-IM6
P-0023857-T01-IM6
P-0025242-T01-IM6
P-0026297-T01-IM6
P-0026297-T02-IM6
P-0028472-T01-IM6
P-0032120-T01-IM6
P-0032496-T01-IM6
P-0032818-T01-IM6
P-0035146-T01-IM6
P-0035147-T01-IM6
P-0035790-T01-IM6
P-0036230-T01-IM6
P-0036297-T01-IM6
P-0038016-T01-IM6
P-0038755-T01-IM6
P-0039704-T01-IM6
P-0039773-T01-IM6
P-0040149-T01-IM6
P-0040284-T01-IM6
P-0040798-T01-IM6
P-0040828-T01-IM6
P-0041346-T01-IM6
P-0042256-T01-IM6
P-0042199-T01-IM6
P-0043045-T01-IM6
P-0043071-T01-IM6
P-0045015-T01-IM6
P-0045458-T01-IM6
P-0047653-T01-IM6


In [259]:
mutAttributionAnnotatedMaf.columns.values

array(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build',
       'Chromosome', 'Start_Position', 'End_Position', 'Strand',
       'Variant_Classification', 'Variant_Type', 'Reference_Allele',
       'Tumor_Seq_Allele1', 'Tumor_Seq_Allele2', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Tumor_Sample_Barcode',
       'Matched_Norm_Sample_Barcode', 'Match_Norm_Seq_Allele1',
       'Match_Norm_Seq_Allele2', 'Tumor_Validation_Allele1',
       'Tumor_Validation_Allele2', 'Match_Norm_Validation_Allele1',
       'Match_Norm_Validation_Allele2', 'Verification_Status',
       'Validation_Status', 'Mutation_Status', 'Sequencing_Phase',
       'Sequence_Source', 'Validation_Method', 'Score', 'BAM_File',
       'Sequencer', 'Tumor_Sample_UUID', 'Matched_Norm_Sample_UUID',
       'HGVSc', 'HGVSp', 'HGVSp_Short', 'Transcript_ID', 'Exon_Number',
       't_depth', 't_ref_count', 't_alt_count', 'n_depth', 'n_ref_count',
       'n_alt_count', 'all_effects', 'Allele', 'Gene', 'Feature',
       'Feature_ty